# Episode 2: Coordinate Transformations

Outline

- cone search
- Use `Quantity` objects to represent measurements with units
- Use Astropy to **convert coordinates** from one frame to another
- Use ADQL keywords `POLYGON` `CONTAINS` `POINT`, to select region in sky
- store results in **FITS** file

## Working with Units

work with physical quantities: $30 ^\circ$
- 2 parts: value and a unit
- value + unit = quantity
- **Astropy** provides tools for including units explicitly in computations, which makes it possible to detect errors before they cause disaters. https://en.wikipedia.org/wiki/Mars_Climate_Orbiter#Cause_of_failure

In [68]:
import astropy.units as u

`u` is an object that contains most common units and all SI units.

use `dir` to list - https://docs.astropy.org/en/stable/units/

In [69]:
dir(u)

['A',
 'AA',
 'AB',
 'ABflux',
 'ABmag',
 'AU',
 'Angstrom',
 'B',
 'Ba',
 'Barye',
 'Bi',
 'Biot',
 'Bol',
 'Bq',
 'C',
 'Celsius',
 'Ci',
 'CompositeUnit',
 'D',
 'DN',
 'Da',
 'Dalton',
 'Debye',
 'Decibel',
 'DecibelUnit',
 'Dex',
 'DexUnit',
 'EA',
 'EAU',
 'EB',
 'EBa',
 'EC',
 'ED',
 'EF',
 'EG',
 'EGal',
 'EH',
 'EHz',
 'EJ',
 'EJy',
 'EK',
 'EL',
 'EN',
 'EOhm',
 'EP',
 'EPa',
 'ER',
 'ERy',
 'ES',
 'ESt',
 'ET',
 'EV',
 'EW',
 'EWb',
 'Ea',
 'Eadu',
 'Earcmin',
 'Earcsec',
 'Eau',
 'Eb',
 'Ebarn',
 'Ebeam',
 'Ebin',
 'Ebit',
 'Ebyte',
 'Ecd',
 'Echan',
 'Ecount',
 'Ect',
 'Ed',
 'Edeg',
 'Edyn',
 'EeV',
 'Eerg',
 'Eg',
 'Eh',
 'EiB',
 'Eib',
 'Eibit',
 'Eibyte',
 'Ek',
 'El',
 'Elm',
 'Elx',
 'Elyr',
 'Em',
 'Emag',
 'Emin',
 'Emol',
 'Eohm',
 'Epc',
 'Eph',
 'Ephoton',
 'Epix',
 'Epixel',
 'Equivalency',
 'Erad',
 'Es',
 'Esr',
 'Eu',
 'Evox',
 'Evoxel',
 'Eyr',
 'F',
 'Farad',
 'Fr',
 'Franklin',
 'FunctionQuantity',
 'FunctionUnitBase',
 'G',
 'GA',
 'GAU',
 'GB',
 'GBa',


**create quantity** by multiplying a value by a unit

In [70]:
angle = 10 * u.degree
type(angle)

astropy.units.quantity.Quantity

The result is a **Quantity** object

In [71]:
print(angle)

10.0 deg


In [72]:
angle

<Quantity 10. deg>

`Quantities` provides a method called `to` that converts to other units.

`angle` in degrees to arcminutes

In [73]:
angle_arcmin = angle.to(u.arcmin)
angle_arcmin

<Quantity 600. arcmin>

- adding `Quantities` will convert units

In [74]:
angle + 30 * u.arcmin

<Quantity 10.5 deg>

if units are not compatible, you get error

In [75]:
angle + 5 * u.kg

UnitConversionError: Can only apply 'add' function to quantities with compatible dimensions

## Selecting a Region

select region of sky?

**Sample Query** from Gaia archive documentation 
 - https://gea.esac.esa.int/archive-help/adql/examples/index.html
 - https://www.cosmos.esa.int/web/gaia-users/archive/writing-queries


 - selects objects in a circular region 
     - centered at (88.8, 7.4)
     - radius of 5 arcmin (0.08333 deg)

In [76]:
cone_query = """SELECT
TOP 10
source_id
FROM \"I/345/gaia2\"
WHERE 1=CONTAINS(
    POINT('ICRS', ra,dec),
    CIRCLE('ICRS', 88.8, 7.4, 0.083))
"""

query uses three keywords, specific to ADQL (not SQL):
- `POINT` ICRS degrees or right ascension and declination. 
    - *International Celestial Reference System*
- `CIRCLE` (ra, dec, radius)
- `CONTAINS` returns 1 or 0

In [77]:
cone_job = tap_service.run_sync(cone_query, language='ADQL')
cone_job.infos

{'QUERY_STATUS': 'OK',
 'PROVIDER': 'CDS',
 'QUERY': 'SELECT TOP 10 source_id FROM "I/345/gaia2" WHERE 1=CONTAINS(     POINT(\'ICRS\', ra,dec),     CIRCLE(\'ICRS\', 88.8, 7.4, 0.083)) '}

In [78]:
cone_results = cone_job.to_table()
cone_results

source_id
int64
3322773724536907008
3322773720244239232
3322773724537891328
3322773720244273024
3322773724537891456
3322773724537890944
3322773724537890048
3322773758899157120
3322773758896634752


### Exercise (5 min)

When you are debugging queries like this, you can use `TOP` to limit the size of the results, but then you still don’t know how big the results will be.

An alternative is to use `COUNT` , which asks for the number of rows that would be selected, but it does not return them.

In the previous query, replace `TOP 10 source_id` with `COUNT(source_id)` and run the query again. How many stars has Gaia identi!ed in the cone we searched?

In [79]:
count_cone_query = """SELECT 
COUNT(source_id)
FROM \"I/345/gaia2\"
WHERE 1=CONTAINS(
  POINT('ICRS', ra, dec),
  CIRCLE('ICRS', 88.8, 7.4, 0.08333333))
"""

count_cone_job = tap_service.run_sync(count_cone_query)
count_cone_results = count_cone_job.to_table()
count_cone_results

COUNT
int64
594


In [80]:
# or just look at value without converting to astropy table
type(count_cone_results)
count_cone_job

<Table length=1>
COUNT
int64
-----
  594

## Getting GD-1 Data

(Price-Whelan and Bonaca 2018) paper


http://www.astroexplorer.org/details/apjlaad7b5f1/eyJrZXlXb3JkcyI6IlByaWNlLVdoZWxhbiIsImF1dGhvciI6IlByaWNlLVdoZWxhbiIsImZyb21ZZWFyIjoyMDE4LCJ0b1llYXIiOjIwMTgsInBhZ2UiOjEsInNob3ciOiIyMDAifQ


- try and reproduce figure 1
- GD-1 stream (Grillmair & Dionatos 2006)

x-axis ($\phi_1$)   $(-100^\circ$ to $20^\circ)$

y-axis (\phi_2)   $(-10^\circ$ to $10^\circ)$

10 million stars in this view

select smaller area 
- -55 to -45
- -8 to 4

## Transforming coordinates

- Astropy has abstracted the **spherical trigonometry** for us
- Equatorial coord. -> Galactic coord. -> GD-1 reference frame

`SkyCoord` object represents sky coordinates relative to a specified reference frame.

 - Here is Betelgeuse in the ICRS frame.     "International Celestial Reference System"

In [81]:
from astropy.coordinates import SkyCoord

ra = 88.8 * u.degree
dec = 7.4 * u.degree
coord_icrs = SkyCoord(ra=ra, dec=dec, frame='icrs')

coord_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (88.8, 7.4)>

`transform_to` function to transform reference frame

IRCS -> Galactic

In [82]:
coord_galactic = coord_icrs.transform_to('galactic')
coord_galactic

<SkyCoord (Galactic): (l, b) in deg
    (199.79693102, -8.95591653)>

Galactic longitude and latitude `l` and `b`

------------------------

Gala library provides tools for galactic dynamics

https://gala-astro.readthedocs.io/en/latest/_modules/gala/coordinates/gd1.html

- GD1Koposov10 is a Heliocentric spherical coordinate system defined by the orbit of the GD-1 stream

In [83]:
from gala.coordinates import GD1Koposov10

gd1_frame = GD1Koposov10()
gd1_frame

<GD1Koposov10 Frame>

In [84]:
coord_gd1 = coord_icrs.transform_to(gd1_frame)
coord_gd1

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    (-94.97222038, 34.5813813)>

### Exercise (10 min)

Find the location of GD-1 in ICRS coordinates.

1. Create a SkyCoord object at 0°, 0° in the GD-1 frame.

2. Transform it to the ICRS frame.

Hint: Because ICRS is a standard frame, it is built into Astropy. You can specify it by name, icrs (as we did with galactic ).

In [85]:
origin_gd1 = SkyCoord(0*u.degree, 0*u.degree, frame=gd1_frame)

origin_gd1.transform_to('icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (200., 59.4504341)>

Notice that the origin of the GD-1 frame maps to `ra=200` , exactly, in ICRS. That is by design.

## Selecting a rectangle

defining a rectangle that emcompasses a small part of GD-1
 - a lot of stars
 - define in GD-1 coordinates

In [86]:
phi1_min = -55 * u.degree 
phi1_max = -45 * u.degree
phi2_min = -8 * u.degree
phi2_max = 4 * u.degree

write a function to build a rectangle
- lower and upper bounds
- returns a list of x and y coordinates
- corners of rectangle starting at lower left corner 
    - clockwise

In [87]:
def make_rectangle(x1, x2, y1, y2):
    """Return the corners of a rectangle."""
    xs = [x1, x1, x2, x2, x1]
    ys = [y1, y2, y2, y1, y1]
    return xs, ys

This returns a *tuple*, a finite ordered list
- list of \phi_1 coord. and a list of \phi_2 coord. 

In [88]:
phi1_rect, phi2_rect = make_rectangle(
    phi1_min, phi1_max, phi2_min, phi2_max)

- coordinates in the Gaia catalog are in the ICRS frame. 
- convert the coordinates from the GD-1 frame to the ICRS frame.
- SkyCoord objects can take lists of coordinates, in addition to single values.

In [89]:
corners = SkyCoord(phi1=phi1_rect, phi2=phi2_rect, frame=gd1_frame)
corners

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.), (-55., -8.)]>

use `transform_to` to convert to ICRS coord

In [90]:
corners_icrs = corners.transform_to('icrs')
corners_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254),
     (146.27533314, 19.26190982)]>

rectangle in one coordinate system is not necessarily a rectangle in another

## Defining a polygon

IN ORDER TO USE THIS POLYGON AS A PART OF AN `ADQL` QUERY
 - we have to convert it to a **string** with a **comma-separated** **list** of coordinates, as in this example:

```
"""
POLYGON('ICRS', 143.65, 20.98, 
        134.46, 26.39, 
        140.58, 34.85, 
        150.16, 29.01)
"""
```

`SkyCoord` provides `to_string`

In [91]:
corners_list_str = corners_icrs.to_string()
corners_list_str

['146.275 19.2619',
 '135.422 25.8774',
 '141.603 34.3048',
 '152.817 27.1361',
 '146.275 19.2619']

- use python `.join` - python string function with spaces between pairs

In [92]:
corners_single_str = ' '.join(corners_list_str)
corners_single_str

'146.275 19.2619 135.422 25.8774 141.603 34.3048 152.817 27.1361 146.275 19.2619'

- replace the spaces with commas.

In [93]:
corners_single_str.replace(' ', ', ')

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

This is something we will need to do multiple times. --- WRITE A FUNCTION

In [94]:
def skycoord_to_string(skycoord):
    """Convert a one-dimenstional list of SkyCoord to string for Gaia's query format."""
    corners_list_str = skycoord.to_string()
    corners_single_str = ' '.join(corners_list_str)
    return corners_single_str.replace(' ', ', ')

In [95]:
sky_point_list = skycoord_to_string(corners_icrs)
sky_point_list

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

## Assembling the query

In [96]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'


In [97]:
query3_base = """SELECT 
TOP 10 
{columns}
FROM \"I/345/gaia2\"
WHERE parallax < 1
  AND bp_rp * 1.0 BETWEEN -0.75 AND 2
"""

- add `WHERE` clause to also select polygon

In [98]:
polygon_top10query_base = """SELECT
TOP 10
{columns}
FROM \"I/345/gaia2\"
WHERE parallax < 1
  AND bp_rp * 1.0 BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS', ra, dec), 
                   POLYGON('ICRS', {sky_point_list}))
"""

FORMAT SPECIFIERS
- `columns`
- `sky_point_list`

In [99]:
polygon_top10query = polygon_top10query_base.format(columns=columns, 
                          sky_point_list=sky_point_list)
print(polygon_top10query)

SELECT
TOP 10
source_id, ra, dec, pmra, pmdec, parallax
FROM "I/345/gaia2"
WHERE parallax < 1
  AND bp_rp * 1.0 BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS', ra, dec), 
                   POLYGON('ICRS', 146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619))



- proof-read ------- before we launch

In [101]:
polygon_top10query_job = tap_service.run_async(polygon_top10query)

In [102]:
polygon_top10query_job.infos

{'QUERY_STATUS': 'OK',
 'PROVIDER': 'CDS',
 'QUERY': 'SELECT TOP 10 source_id, ra, dec, pmra, pmdec, parallax FROM "I/345/gaia2" WHERE parallax < 1   AND bp_rp * 1.0 BETWEEN -0.75 AND 2    AND 1 = CONTAINS(POINT(\'ICRS\', ra, dec),                     POLYGON(\'ICRS\', 146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619)) '}

In [103]:
polygon_top10query_results = polygon_top10query_job.to_table()
polygon_top10query_results

source_id,ra,dec,pmra,pmdec,parallax
,deg,deg,mas / yr,mas / yr,mas
int64,float64,float64,float64,float64,float64
621490014069058944,146.25027906702,19.28661719637,1.247,-7.877,-0.3693
621492075653372160,146.32228572922,19.34449004493,-2.399,-2.333,0.7228
621492178732589952,146.31743441943,19.36186878516,6.749,0.11,0.9157
621493106445518720,146.21614836191,19.31720803041,2.248,-2.692,0.5535
621493209524733312,146.20333869328,19.31501717901,-2.752,-0.35,-0.1249
621493312603950336,146.20137461153,19.32281010865,-7.993,-0.599,0.4103
621494309036367360,146.14573420631,19.35364591347,-2.07,-7.448,0.173
621494412115584000,146.16288826603,19.36289494846,-2.77,-6.444,0.2028


- remove `TOP 10`

In [104]:
polygon_query_base = """SELECT
{columns}
FROM \"I/345/gaia2\"
WHERE parallax < 1
  AND bp_rp * 1.0 BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS', ra, dec), 
                   POLYGON('ICRS',{sky_point_list}))
"""

- much bigger query will take longer to run

In [105]:
polygon_query = polygon_query_base.format(columns=columns, 
                          sky_point_list=sky_point_list)
print(polygon_query)

SELECT
source_id, ra, dec, pmra, pmdec, parallax
FROM "I/345/gaia2"
WHERE parallax < 1
  AND bp_rp * 1.0 BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT('ICRS', ra, dec), 
                   POLYGON('ICRS',146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619))



RESULT IS A BIGGER QUERY THAT WE HAVE RUN SO MAY TAKE MORE TIME.
- MINE RAN ON THIRD TRY

In [107]:
polygon_job = tap_service.run_async(polygon_query)

In [108]:
polygon_job.infos

{'QUERY_STATUS': 'OK',
 'PROVIDER': 'CDS',
 'QUERY': 'SELECT source_id, ra, dec, pmra, pmdec, parallax FROM "I/345/gaia2" WHERE parallax < 1   AND bp_rp * 1.0 BETWEEN -0.75 AND 2    AND 1 = CONTAINS(POINT(\'ICRS\', ra, dec),                     POLYGON(\'ICRS\',146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619)) '}

In [109]:
polygon_results = polygon_job.to_table()

In [110]:
len(polygon_results)

140339

- There are more than 100,000 stars in this polygon, but that’s a manageable size to work with.

## Saving results

- Astropy `Table` objects provide `write`

- overwrite

In [111]:
filename = 'gd1_results.fits'
polygon_results.write(filename, overwrite=True)

- filename.fits - FITS - format preserves the metadata

- USE `getsize` to check write worked

In [112]:
from os.path import getsize

MB = 1024 * 1024
getsize(filename) / MB

6.4324951171875

## Summary

 - use units
 - use `format` function -- less error-prone
 - developed query incrementally
 - complex queries select polygon region
 - save data to local file after query works
     - download and save to .fits file
